#◢ DeOldify - Colorize your own photos!

####**Credits:**

Special thanks to:

Matt Robinson and María Benavente for pioneering the DeOldify image colab notebook.  

Dana Kelley for doing things, breaking stuff & having an opinion on everything.



---


#◢ Verify Correct Runtime Settings

**<font color='#FF000'> IMPORTANT </font>**

In the "Runtime" menu for the notebook window, select "Change runtime type." Ensure that the following are selected:
* Runtime Type = Python 3
* Hardware Accelerator = GPU 


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import torch

if not torch.cuda.is_available():
    print('GPU not available.')

#◢ Git clone and install DeOldify

In [3]:
!git clone https://github.com/jantic/DeOldify.git DeOldify 

Cloning into 'DeOldify'...
remote: Enumerating objects: 1940, done.
remote: Total 1940 (delta 0), reused 0 (delta 0), pack-reused 1940
Receiving objects: 100% (1940/1940), 69.28 MiB | 37.05 MiB/s, done.
Resolving deltas: 100% (841/841), done.


In [4]:
cd DeOldify

/content/DeOldify


#◢ Setup

In [5]:
!pip install -r colab_requirements.txt

     |████████████████████████████████| 215kB 16.5MB/s 
     |████████████████████████████████| 133kB 30.5MB/s 
     |████████████████████████████████| 1.8MB 42.9MB/s 
     |████████████████████████████████| 6.4MB 55.4MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
  Created wheel for ffmpeg: filename=ffmpeg-1.4-cp36-none-any.whl size=6085 sha256=ce310b582f36d4ec711216ba1f4cc87fc6aa322bae997acfdb04a4ef3716fc61
  Stored in directory: /root/.cache/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built ffmpeg
  Found existing installation: fastai 1.0.59
    Uninstalling fastai-1.0.59:
      Successfully uninstalled fastai-1.0.59
  Found existing installation: jsonschema 2.6.0
    Uninstalling jsonschema-2.6.0:
      Successfully uninstalled jsonschema-2.6.0


In [0]:
import fastai
from deoldify.visualize import *

torch.backends.cudnn.benchmark = True

In [7]:
!mkdir 'models'
!wget https://www.dropbox.com/s/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth?dl=0 -O ./models/ColorizeArtistic_gen.pth

--2020-01-06 18:02:42--  https://www.dropbox.com/s/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.1, 2620:100:6019:1::a27d:401
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth [following]
--2020-01-06 18:02:42--  https://www.dropbox.com/s/raw/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9fce4208f156969e316b871f79.dl.dropboxusercontent.com/cd/0/inline/AvrRc8vWeQs7sI3GBVCLjJcRqGMUnhv2XBwoTB_sLJCLItDr75og_7SmfeUbm5eJGMG0oBbskc13qtjnnmMIl4d4b9K3BGl8pY9mnPHzG6onJw/file# [following]
--2020-01-06 18:02:43--  https://uc9fce4208f156969e316b871f79.dl.dropboxusercontent.com/cd/0/inline/AvrRc8vWeQs7sI3GBVCLjJcRqGMUnhv2XBwoTB_sLJCLItDr75og_7SmfeUbm5eJGMG0oBbskc13qtjnnmMIl4d4b9K3BG

In [8]:
colorizer = get_image_colorizer(artistic=True)

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:442: UserWarning: Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.
  warn("Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.")
/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:445: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")
Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 96.8MB/s]
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It

In [9]:
!pip install ImageHash

     |████████████████████████████████| 296kB 55.5MB/s 


#◢ Instructions

### source_url
Type in a url to a direct link of an image.  Usually that means they'll end in .png, .jpg, etc. NOTE: If you want to use your own image, upload it first to a site like Imgur. 

### render_factor
The default value of 35 has been carefully chosen and should work -ok- for most scenarios (but probably won't be the -best-). This determines resolution at which the color portion of the image is rendered. Lower resolution will render faster, and colors also tend to look more vibrant. Older and lower quality images in particular will generally benefit by lowering the render factor. Higher render factors are often better for higher quality images, but the colors may get slightly washed out. 

#### How to Download a Copy
Simply right click on the displayed image and click "Save image as..."!

## Pro Tips

You can evaluate how well the image is rendered at each render_factor by using the code at the bottom (that cell under "See how well render_factor values perform on a frame here"). 

## Troubleshooting
If you get a 'CUDA out of memory' error, you probably have the render_factor too high.

#◢ Colorize!!

In [10]:
%%time

# path to CSV file which lists the source photo URLs to colorize
photo_csv = '/content/gdrive/My Drive/SigneBrander/photos-since-20191123.csv'

import imagehash
import io

def fetch_image(image_url):
  response = requests.get(image_url, stream=True)
  imgdata = io.BytesIO(response.content)
  return PIL.Image.open(imgdata).convert('RGB')

def colorize(orig, render_factor=35):
  origpath = 'orig.png'
  orig.save(origpath)
  colorized = colorizer.get_transformed_image(path=origpath, render_factor=render_factor)
  return colorized


import csv
with open(photo_csv) as photoscsv:
  reader = csv.reader(photoscsv)
  for row in reader:
    tweet_id, text, tweet_url, image_url = row
    #if image_url == 'https://pbs.twimg.com/media/EKDvvu0XkAAj03-.jpg:large':
    #  continue  # skip first
    orig = fetch_image(image_url)
    imghash = imagehash.dhash(orig)
    if str(imghash) == '0000000000000000':
      print("couldn't fetch image {} ({})".format(tweet_id, text))
      continue
    imgpath = '/content/gdrive/My Drive/SigneBrander/colorized/{}.png'.format(imghash)
    if os.path.exists(imgpath):
      print("image {} {} ({}) already found".format(tweet_id, imghash, text))
      continue
    print("colorizing {} {}: {}".format(tweet_id, imghash, text))
    colorized = colorize(orig, 32)
    colorized.save(imgpath)
    #break

  

image 1198221105481289728 f79f9fe7e8c60ecf (Näkymä Aleksanterinkadulta, vasemmalla rohdoskauppa (1907, Kaupunginmuseo, #helsinki)) already found
image 1198266404811747335 6f69491c07250505 (kuvanveistäjä Walter Runeberg ateljeessaan (1988, Museovirasto)) already found
image 1198311702145765377 a4e448c081686470 (Vantaanjoki (1907, Kaupunginmuseo, #helsinki)) already found
image 1198492895990472704 de4d4122130371ff (Pasilan huvila-alue (1912, Kaupunginmuseo, #helsinki)) already found
image 1198583493728579596 90868d2c6cec9c9c (Vanha rautatieasema (1909, Kaupunginmuseo, #helsinki)) already found
image 1198628792442507265 f878362763686034 (Suitian kartano (1910, Museovirasto)) already found
image 1199217672292110336 a0dcedebece0d2e8 (Havis Amanda -suihkukaivo Kauppatorilla paljastustilaisuuden jälkeen (1908, Kaupunginmuseo, #helsinki)) already found
image 1199262970263613442 80102080c3c0a130 (Sauna Marie-Bad (1913, Kaupunginmuseo, #helsinki)) already found
image 1199308272668495873 f0f0b0b0